## QCoDeS Training for Developers

Welcome to the training about QCoDeS targeted for developers. After this training, developers should have more knowledge about QCoDeS which will bring them confidence with contributing to it.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

### Approach

- Don't bring too much relevant background, only small absolutely necessary portions
- Quickly/slightly refresh the user-side view on QCoDeS, and get close to code
- Describe features/concepts from ideas, but get close to code very quickly

### Execution plan for presenter

- Before:
  - Have qcodes code at hand in your IDE https://github.com/qcodes/qcodes
  - Run the training Jupyter notebook, and have it at hand https://github.com/astafan8/Qcodes/blob/training-for-devs/docs/examples/QCoDeS%20Training%20for%20Developers.ipynb
  - Familiarize yourself with the following info from the docs:
    - Front page https://qcodes.github.io/Qcodes/index.html
    - 15 minutes of QCoDeS https://qcodes.github.io/Qcodes/examples/15_minutes_to_QCoDeS.html
    - `Parameter`s
      - overview1 https://qcodes.github.io/Qcodes/examples/Parameters/Parameters.html
      - overview2 https://qcodes.github.io/Qcodes/examples/writing_drivers/Creating-Instrument-Drivers.html#What%E2%80%99s-a-Parameter? - only this section
      - API docs https://qcodes.github.io/Qcodes/api/parameters/parameter.html#module-qcodes.instrument.parameter
      - `ParameterWithSetpoints` https://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html
    - Performing measurements https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html
    - dataset notebooks
      - the class https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html
      - data access https://qcodes.github.io/Qcodes/examples/DataSet/Accessing-data-in-DataSet.html
- During:
  - Go through notebook in 3 parts, have 5 min breaks, 5 min reserve beginning, 10 min reserve at the end
  - Have a time keeper
  - Let ask questions at specific moments but very frequently to feel interactive
- After:
  - Port notebook to qcodes docs ASAP
  - Sessions on some parts if requested

### Timings

- 20min - intro
  - buffer
  - org intro
  - "qcodes? ah?" section
- 1h 35m - "i wanna measure" section
  - 15min - Station
  - 30min - Instrument
  - 30min - Parameter
  - 20min - Snapshot
- 1h 30m - "i wanna store" section
  - 10min - general topics
  - 15min - Measurement
  - 20min - DataSaver
  - 15min - DataSet
  - 30min - SQLite database
- 20min - outro
  - "i want more" section
  - buffer

### Imports

In [1]:
import qcodes

from typing import Optional, Sequence, Dict
from pprint import pprint

import numpy

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-miasta\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-miasta\.qcodes\logs\200423-21448-qcodes.log


## QCoDeS? ah?

What is it? What are it's goals, and users? And why?

### "The It"

**Q.** uantum **Co** penhagen **De** lft **S** ydney:

- data acquisition framework
- modular
- python-based
- open-source

### Goal

**Common** framework for physics experiments:

- users need to write only their-own-experiment-specific code
- physics experiments can take advantage of modern software and best practices
- code can and should be contributed back to the framework
- the process of moving between teams or labs, and of setting up a new experiment is streamlined
- new students don’t need to spend a long time learning software in order to participate in experiments

### Users

- people doing measurements:
  - academics: e.g. scientists and students
  - corporates: e.g. experimentalists, engineers, hardware manufacturers
- worldwide

### Driven by

- Microsoft, largely
- worldwide comminity, growing

### Development principles

- Robust code - as opposed to what users are used to from other measurement frameworks
- Respect backwards compatibility - don't break user's code that "works"!
- Modularity - extend-existing, not go-off-and-write-your-own
- User facing API - list in docs, devs can use everything as appropriate
- All of the above is basically maintatining users' trust
- Carefully scoping problems

## I wanna MEASURE

How do I measure things? How are measurements enabled via QCoDeS?

We will be looking into the following pyramid of objects and concepts:
- Station - the ultimate bucket
  - Instruments - and channels
    - Parameters - many kinds
- Snapshot - capturing state

### Station

``Station`` is just a bucket for components such as instruments, parameters, etc.

Speaking more strictly, those components should be snapshot-able (subclasses of ``Metadatable``), but more about this in the section about the snapshot.

``Station`` can be created with python code and/or using YAML configuration file. [Read on Station, its interface, YAML config, and more here](http://qcodes.github.io/Qcodes/examples/Station.html).

In [2]:
from qcodes.station import Station

#### Default station

Notice the ``default=True`` argument of the ``Station`` - it controls wether to store the initialized ``Station`` object into an attribute.

Using ``Station.default`` as a fallback in other objects in QCoDeS is very common, as it is extremely uncommon that more than one ``Station`` is initiated in one python session.

In [3]:
station1 = Station()  # default=True

assert Station.default is station1  # the ``default`` is set

station2 = Station()  # default=True

assert Station.default is not station1
assert Station.default is station2  # the ``default`` changed

station3 = Station(default=False)

assert Station.default is not station3  # the ``default`` is not changed

### Instruments

Instrument conceptually represents hardware that one needs to talk to in order to perform measurements.

The concept can be expanded to also include "instruments" that are:

- not remotly controllable but it's important to record their state
- exposing a more measurement-specific interface while aggregating actual instruments

#### Drivers

Concrete implementation of classes and abstracations provided by QCoDeS for particular piece of hardware is commonly referred to as ``drivers`` or ``QCoDeS drivers`` for that piece of hardware.

From a real conversation:

> - Shall we buy this useful, powerful, and expensive measurement equipment?
> - Hmm... Is there a QCoDeS driver for it?

#### Base class(es)

From coding perspective, instruments in QCoDeS are subclasses of ``Instrument`` and ``InstrumentBase`` from ``qcodes.instrument.base``.

``InstrumentBase`` - defines "interface":

- has a name
- is snapshot-able (subclass of ``Metadatable``)
- serves as a bucket for ``Parameter``s
- may include snapshot-able "submodules"
  - navigation via ``root_instrument``, ``parent``, ``ancestors``
- instance-bound logger
- convenient dot-notation access to it's parameters and submodules (thanks to ``DelegateAttributes``)
- convenient ``[]``-notation access to them as well

``Instrument`` - accounts for communication:

- subclass of ``InstrumentBase``
- hides communication with hardware via ``ask``/``write``
- implements global registry of instrument instances
- ``IDN``, ``connect_message``

#### Submodules

Submodules allow for logical grouping of functionality within an instrument. For example, [look at the trigger, sample, and display submodules of Keysight 344xxA driver](https://qcodes.github.io/Qcodes/examples/driver_examples/Qcodes%20example%20with%20Keysight%20344xxA.html#Parameters-and-methods-overview) which allow to conveniently group parameters and methods without polluting the root namespace of the driver.

To add a submodule to an instrument, use it's ``add_submodule`` method (usually in instrument's ``__init__``).

``InstrumentBase`` promises to include snapshots of it's submodules in its own snapshot.

How to implement submodules may not be clear:
- the current implementation suggests that any ``Metadatable`` object will work
- the typing of ``add_submodule`` suggests using/subclassing ``InstrumentBase`` in order to take full advantage of the ``InstrumentBase``s interface for the submodule
- most commonly used/cubclasses is ``InstrumentChannel`` from ``qcodes.instrument.channel`` because
  - it takes care of correct naming and 
  - forwards communication to the parent instrument

#### Channels

Channels are most popular submodules of instruments. Why? Think of a multi-channel voltage source kind of instrument.

QCoDeS provides ``InstrumentChannel`` class that takes care of correct naming and forwards communication to the parent instrument. ``InstrumentChannel`` can be added as submodules to the instrument.

Moreover, QCoDeS provides ``ChannelList`` class. In short, it allows to group channels that are of the same class, and implements convenient access to common parameters and functions of those channels. See the difference between the following ways of setting voltage parameter of all channels to ``0``:
```python
my_voltage_source.channels.voltage(0)
```
versus
```python
for channel in my_voltage_source.channels:
    channel.voltage(0)
```
or even worse
```python
channel_names = [...]  # I just happen to know them
for channel_name in channel_names:
    channel = my_voltage_source[channel_name]
    channel.voltage(0)
```

#### Hardware communication

``Instrument`` class suggests the most frequent use case of how communication with hardware is being performed:
- ``write(cmd: str) -> None``
- ``ask(cmd: str) -> str``

Subclasses of ``Instrument`` should implement concrete communication by overriding ``write_raw`` and ``ask_raw``.

A lot of QCoDeS abstractions are built with this string-command-based communication model in mind, especially ``Parameter``.

##### ``VisaInstrument``

A great example is ``VisaInstrument`` from ``qcodes.instrument.visa`` - it implements ``write_raw`` and ``ask_raw`` on top of ``pyvisa`` library that provides a python wrapper over implementations of VISA (Virtual Instrument Software Architecture) API.

Here's a schematic of the layering, from high, user-facing, to low:
```
- MyInstrumentDriver . parameter/method
- VisaInstrument     . ask/write
- Instrument         . ask_raw/write_raw
- pyvisa .. Resource . query/write
- Actual hardware communication layer, e.g. LAN, USB
  implemented by a VISA driver (e.g. National Instruments, pyvisa-py, etc)
```

The command API of the most VISA instruments conforms to SCPI (Standard Commands for Programmable Instruments) standard which makes QCoDeS driver development for those instruments extremely easy.

##### Other

QCoDeS does **not** restrict interacting with hardware only with this communication model. An good example of this are instruments which use some specific ``.dll`` library, see drivers for [Alazar high-speed digital acquisition cards](https://github.com/QCoDeS/Qcodes/blob/master/qcodes/instrument_drivers/AlazarTech/ATS.py#L68), [Vaunix step attenuator](https://github.com/QCoDeS/Qcodes_contrib_drivers/blob/c007532d48dfc035a9e2f03167606c16c42d7f77/qcodes_contrib_drivers/drivers/Vaunix/LDA.py#L59), etc.

In that case, ``write_raw``/``ask_raw`` are usually left as raising ``NotImplementedError``, and parameters and methods of the instrument driver don't use them at all.

#### Registry of instances

QCoDeS does not allow creation of an instrument with the same ``name``. Why? - To enforce that there is **only one** connection to an instrument. AND there is a way to distinguish between instruments within a measurement context.

##### No instruments with same name

Let's try to create two instruments with the same name:

In [4]:
from qcodes.instrument.base import Instrument

foo_called_foo = Instrument('foo')

In [5]:
try:
    bar_called_foo = Instrument('foo')
except:
    import traceback
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-5-1e60ed53d4d7>", line 2, in <module>
    bar_called_foo = Instrument('foo')
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 431, in __init__
    self.record_instance(self)
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 567, in record_instance
    raise KeyError('Another instrument has the name: {}'.format(name))
KeyError: 'Another instrument has the name: foo'


In order to get rid of the exception, the user needs to ``close`` one of the instruments:

In [6]:
foo_called_foo.close()

In [7]:
bar_called_foo = Instrument('foo')
bar_called_foo 

<Instrument: foo>

##### Fool proof?

This may not be the optimal mechanism.

For example, one can fool the mechanism by using two different names but the same VISA address:
```python
foo_at_address1 = VisaInstrument('foo', address='address1')
bar_at_address1 = VisaInstrument('bar', address='address1')
# DISCLAIMER: only tried with pyvisa-sim, not a real VISA driver
```
in which case it's now up to the "connectivity backend" (in this case, VISA) to deal with having multiple "connections" to the same piece of hardware.

##### Behind the scenes

Behind the scenes QCoDeS maintains a registry of instrument instances by their names. The scope of the registry is the python process.

``Instrument`` class maintains an ``_all_instruments`` name-to-weak-reference dictionary in a class attribute.

Each ``Instrument`` subclass maintains its own ``_instances`` list of weak references to instances of **only that (sub)class** as in a class attribute (and as ``instances`` method).

``Instrument`` class provides a few class methods for navigating in the registry: ``instances``, ``find_instrument``, ``exists``, ``is_valid``. The most popular however are ``close_all`` method and ``find_or_create-instrument`` function.

##### ``find_or_create_instrument``

There is a ``find_or_create_instrument`` convenience function that allows to conveniently find an existing instrument, create it if it doesn't exists, or re-create it if it does exist:

In [8]:
from qcodes.instrument.base import find_or_create_instrument

In [9]:
# Example of Find
foo = find_or_create_instrument(Instrument, 'foo')
assert foo is bar_called_foo

Connected to: None foo (serial:None, firmware:None) in 0.06s


In [10]:
# Example of Recreate
foo = find_or_create_instrument(Instrument, 'foo', recreate=True)
assert foo is not bar_called_foo

In [11]:
# Example of Create
class MyInstrument(Instrument):
    pass

foo = find_or_create_instrument(MyInstrument, 'bar')

##### ``close_all``

Another useful API is ``close_all`` - it closes all instrument instances.

In [12]:
print('before closing all:', Instrument._all_instruments)

Instrument.close_all()

print('after closing all:', Instrument._all_instruments)

before closing all: {'foo': <weakref at 0x0000026CA475E0E8; to 'Instrument' at 0x0000026CA473E2C8>, 'bar': <weakref at 0x0000026CA47668B8; to 'MyInstrument' at 0x0000026CA472DC08>}
after closing all: {}


##### And ``Station``?

``Station`` is aware of the instrument instance registry. It exposes convenient API which is station-scoped version of the above-mentioned ``Instrument`` APIs:

- ``.load_instrument(.., revive_instance=False, ..)`` - for ``find_or_create_instrument``
- ``.close_and_remove_instrument(..)`` - for ``Instrument.close``
- ``.close_all_registered_instruments(..)`` - for ``Instrument.close_all``

### Parameters

Parameters in QCoDeS serve mostly two purposes:

- values that can be **set** or **measured** via an instrument, e.g. voltage, current, time trace
- "settings" of an instrument, e.g. range, precision, NPLCs

**NOTE** that context of an experiment/measurement is important. The same parameter may serve either of the purposes depending on the context. Think of an offset of a sine wave - in one experiment that will be just a setting, in another it will "set" to different values (e.g. swept over).

In QCoDeS, parameter is modeled as an object that is:

- representing a value (in broad sense)
- and metadata about that value,
- and that value can, or can't, be ``set`` and ``get``

#### Base class(es)

The main base class for parameters in QCoDeS is ``Parameter`` from ``qcodes.instrument.parameter``.

All parameters should be created/instantiated from ``Parameter``, and more sophisticated parameters should be implemented as subclasses ``Parameter``.

However, ``Parameter`` inherits from a ``_BaseParameter``. The separation between the two may not be clear from the code, yet let's try to see it below:

``_BaseParameter`` - the "meat":

- has a name
- implements ``set``/``get`` methods (and ``__call__`` convenience)
  - via wrapping ``get_raw``/``set_raw`` that are to be overridden
- built-in ``set``/``get`` behavior controls:
  - value validators // ``get_parser``, ``set_parser`` // ``val_mapping``
  - ``scale``, ``offset`` // ``step`` // ``inter_delay``, ``post_delay``
- has ``cache`` of the parameter value (also ``get_latest``)
- may be aware of the instrument it belongs to
- is snapshot-able (subclass of ``Metadatable``), and flags to control it

``Parameter`` - the important extension:

- subclass of ``_BaseParameter``
- adds ``label``, ``unit``, ``docstring``
- enables strings, callables, ``None`` and ``False`` as ``set``/``get``
- interfaces for the legacy ``Loop`` feature

#### Name, label, unit, ...

Metadata is a very important part of the parameter. It allows to reason about the meaning of its value, and ``set``/``get`` commands (if any). Hence, it is important to assign good names, labels, units, and a docstring.

``name`` is NOT supposed to be changed after parameter's initialization.

``label`` and ``unit`` are plain attributes of ``Parameter`` and are not recommended to be changed after initialization.

However, it is tempting to adjust ``label``/``unit`` sometimes. When a parameter represents some sort of an acquisition buffer of an instrument which can hold both, for example, voltage and current values, it'd be great to change ``label``/``unit`` to ``"Voltage"``/``"V"`` when the instrument is aquiring voltage, and to ``"Current"``/``"A"`` when it's aquiring current. Note that unfortunately, the docstring of the parameter (the ``__doc__``) will still use ``label``/``unit`` that the parameter was initiated with. That is to say that not all ``Parameter`` features are implemented with change-ability of ``label``/``unit`` in mind.

Below a few parameters are created, and their final docstring is printed.

In [13]:
from qcodes.instrument.parameter import Parameter

delay = Parameter(
    'delay',
    label='Trigger Delay',
    unit='s',
    docstring='Delay between the trigger signal and the first measurement'
)

voltage = Parameter(
    'voltage',
    label='Voltage',
    unit='V',
    docstring='Measured voltage'
)

In [14]:
print(delay.__doc__)

Delay between the trigger signal and the first measurement

Parameter class:

* `name` delay
* `label` Trigger Delay
* `unit` s
* `vals` None


In [15]:
print(voltage.__doc__)

Measured voltage

Parameter class:

* `name` voltage
* `label` Voltage
* `unit` V
* `vals` None


#### ``add_parameter`` of ``Instrument``

Parameters are usually conveniently grouped into instruments or their submodules. This is why ``InstrumentBase`` implements a convenient ``add_parameter`` method - it's purpose is to:

- initiate a parameter of optionally provided class with provided arguments,
- ensure that the instrument instance is passed to the parameter, and
- add the initiated parameter to instruments collection of parameters (in ``.parameters``).

It is important that ``add_parameter`` ensures to pass to the instrument instance to the parameter. Thanks to this:
- the parameter can use the instrument instance via its ``.instrument`` and ``root_instrument`` properties
- parameter can leverage the instrument's ``write``/``ask`` methods for communication with the hardware
- parameter's ``full_name`` gets prefixed by the instrument's name (or, more broadly, by the channel's or submodule's name)

#### ``set`` and ``get``

``set`` and ``get`` methods of a parameter are it's primary interface for setting and getting it's value:

* ``.set(value: ParamDataType) -> None``
* ``.get() -> ParamDataType``

(``ParamDataType`` is an alias for ``Any``, and is used mostly for readability.)

Actual signatures are more flexible - ``kwargs`` for both and even ``args`` for ``get``:
* ``.set(value: ParamDataType, **kwargs: Any) -> None``
* ``.get(*args: Any, **kwargs: Any) -> ParamDataType``

Usage of ``args`` and ``kwargs`` is discouraged in order not to pollute the simplicity of the ``Parameter``'s interface.

Where does the ``value`` come from or get sent to? It can be:

- some instrument - communication with hardware
- memory - value stored in memory, and is just useful in a particular context
- **anything** - ``set``/``get`` are just callables, they can really do anything about the value :)

``set`` and ``get`` methods are not explicit bound methods of a parameter. They are created during ``__init__`` by wrapping other methods which concretely implement what needs to happen when a value is being ``set`` or ``get``.

The ways to provide these concrete implementations for wrapping into actual ``set``/``get`` methods are described below.

##### ``set_raw`` and ``get_raw``

The primary way of implementing behavior of ``set``/``get`` is to override ``set_raw``/``get_raw`` methods of ``Parameter`` (well, actually ``_BaseParameter``). Behind the scenes, calling ``set``/``get`` will forward the call to ``set_raw``/``get_raw``.

Below we implement a trivial parameter that accesses a variable in outer scope. (It is convenient to use for demonstration, but never do this in production :) )

In [16]:
precious_value = 42

class PreciousParameter(Parameter):
    def set_raw(self, value):
        global precious_value
        precious_value = value

    def get_raw(self):
        global precious_value
        return precious_value

In [17]:
my_precious = PreciousParameter(
    'my_precious',
    label='My Precious Value',
    unit=''  # precious things are priceless, unitless
)

# Value gotten via the parameter should equal to ``precious_value``
parameter_value = my_precious.get()
assert parameter_value == precious_value

# Setting parameter value should be reflected in ``precious_value``
my_precious.set(66)
assert precious_value == 66

# Getting parameter value again should still return the new ``precious_value``
parameter_value = my_precious.get()
assert parameter_value == precious_value

##### ``set_cmd`` and ``get_cmd``

Historically, more flexible and convenient ways of defining parameter's ``set``/``get`` methods behavior were needed. This is how ``Parameter`` (not ``_BaseParameter``) got two interesting arguments - ``set_cmd`` and ``get_cmd``. Let's go through the "values" that can be supplied with these arguments.

###### ``False`` - non-get-able/non-set-able parameter

If a ``Parameter`` is initialized with the set/get commands set to ``False`` then its not settable/gettable. This means that the instance will not have the ``set``/``get`` methods respectively.

* Having a set-only or a get-only parameter may be useful
  * for example for measurement parameters, or settings/characteristics that an instrument calculates internally
* If a parameter is set-only or get-only, it is still possible to access its cache which is useful when implementinga driver.
* It is rarely useful to have a parameter that can't be both set and gotten.
* ``set_cmd=False`` is ``Parameter``s default, likely due to frequent occurence of get-only instrument parameters.

###### ``None`` - "memory parameter", store/take value in/from memory

If a ``Parameter`` is initialized with commands set to ``None``, then parameter's internal cache will be the only place where the value is stored.

* It is possible to pass only one of the ``set_cmd``/``get_cmd`` as ``None``
  * The reason this works is that parameter's cache is used anyway to store the value,
    so any additional behavior of ``set``/``get`` is an "extra".
  * This can be useful - for example, if an instrument permits to only get or only set some settings,
    it may be convenient to rely on parameter's cache for the not-permitted operation.
* ``get_cmd=None`` is ``Parameter``s default, likely due to frequent occurence of set-only instrument parameters.

###### String ``Command``s - convenient for SCPI (all text-based) instruments

Text-based communication is very popular among instruments. Most of them even conform to the SCPI standard of the text-based instrument API.

Now, let's look at what pattern is frequently arises because of this - let's try to implement a driver for a VISA instrument that uses text-based communication with one parameter.

In [18]:
from qcodes.instrument.visa import VisaInstrument
from qcodes.instrument.parameter import Parameter


class Meter(VisaInstrument):
    # This will be a simple instrument that can measure
    # 'a feeling of temperature' and return a string.
    # It also allows us to specify a range which increases
    # its vocabulary of temperature feelings.

    def __init__(self, name, address):
        # skipping many other arguments for simplicity
        super().__init__(name, address)
        
        self.add_parameter('measure', parameter_class=MeasureParameter)
        self.add_parameter('range', parameter_class=RangeParameter)

In [19]:
class MeasureParameter(Parameter):
    # In order not to make this example too complex,
    # let's assume that the instrument returns a string
    # value, say, "HOT"/"WARM"/"COOL"/"COLD"
    def get_raw(self):
        return self.instrument.ask('MEASURE?')


class RangeParameter(Parameter):
    # In order not to make this example too complex,
    # let's assume that the instrument specifies the range
    # via a string value, say, "OUTDOORS"/"INDOORS"

    def set_raw(self, value):
        return self.instrument.write(f'RANGE {value}')

    def get_raw(self):
        return self.instrument.ask('RANGE?')

Notice that both parameters use instrument's ``ask``/``write`` methods. Moreover, both of the parameter classes just implement usage of different string commands, they do nothing more.

For such a simple instrument, this is not that bad, yet still quite some boilerplate in order to create just two parameters with the only purpose of providing a command string. Imagine a more realistic driver - that one would usually have 10s of parameter, hence more boilerplate code with this approach.

Let's compare the above code with the following code that uses ``set_cmd``/``get_cmd``:

In [20]:
class SmartMeter(VisaInstrument):
    # Same as ``Meter`` but fewer lines of code

    def __init__(self, name, address):
        super().__init__(name, address)

        self.add_parameter('measure', get_cmd='MEASURE?')
        self.add_parameter('range', set_cmd='RANGE {}', get_cmd='RANGE?')

This is the same functionality but in much more concise code.

So, how do ``set_cmd``/``get_cmd`` work? ``Command`` object from ``qcodes.utils.command`` encapsulates the functionality.

Here is how ``set_cmd``/``get_cmd`` are processed:

- ``Parameter`` notices that ``set_cmd``/``get_cmd`` is not a ``None`` or ``False``
- ``Parameter`` creates a ``Command`` object
- the value of ``set_cmd``/``get_cmd`` is passed to ``Command``
- parameter's instrument's ``write``/``ask`` methods are also passed to ``Command``
- The created ``Command`` object is callable, hence can be assigned as ``set``/``get`` method of the ``Parameter``
- ``Command`` treats the passed ``set_cmd``/``get_cmd`` string as a python format string (thus enabling value-setting kind of commands)
- When called ``Command`` formats the command string with given value, and the formatted string is passed to the previously-given ``write``/``ask`` instrument methods.

String-based no-value and single-value commands are very common among instruments, and thanks to ``set_cmd``/``get_cmd`` can be easily implemented.

Note that historically ``Command`` class itself is very flexible, and not all of it's features are used.

###### Callable ``Command``s

Sometimes string commands are not enough to implement the behavior of parameter's ``set``/``get`` methods. At the same time, creating a separate subclass of ``Parameter`` and implementing its ``set_raw``/``get_raw`` seems an overkill.

Solution to that is to pass callables to ``set_cmd``/``get_cmd`` (functions or methods). This is one of the *useful* flexibilities of the ``set_cmd``/``get_cmd`` and ``Command`` class.

One of the most frequent use cases for callable ``set_cmd``/``get_cmd`` is when setting a parameter induces changes in other parameters on the instrument such we need to request those values explicitly in order to make sure that the driver correctly represents the state of the instrument.

#### ``set_to``

An important convenience that parameters implement is a ``set_to`` context manage that allows to set the parameter value to a given value within the context, and then revert its value to the one that the parameter had before the context.

This feature is important because it removes the need for users remember to revert the parameter to its original value (if relevant). At a larger scale, this feature provides convenience for putting an instrument in a desired state and automatically reverting from it, in case the state is represented by a *number* of parameters.

The usage of the context manager looks like this:
```python
param.set(original_value)
with param.set_to(new_value):
    print(param())
    # here `param`'s value is `new_value`
print(param())
# here `param`'s value is again `original_value`
```

This is possible because parameter maintains a cache of it's value, hence it is aware of its "original" value, and hence the content manager can revert the parameter to it.

If a parameter has never been set or gotten before using the ``set_to`` context, if possible, it's value will be requested by calling parameter's ``get``. This is thanks to the smartness of the parameter's cache.

#### ``cache`` and ``get_latest``

Historically, the feature request was to provide a way to obtain parameter's value without requesting it from the instrument again. That's when ``get_latest`` was born.

``Parameter.get_latest()`` (or ``Parameter.get_latest.get()``) returns the value that has been received at the **latest** ``Parameter.get()`` call. That value would be stored internally in ``Parameter`` object.

Later, with more use cases, ``cache`` was introduced. It subsumes features of ``get_latest`` and extends them. From coding perspective, ``cache`` is also a cleaner encapsulation.

``cache`` is an attribute of ``_BaseParameter`` that holds an instance of ``_Cache`` class that is aware of it's parent parameter. ``get_latest`` interface remains for backwards compatibility, and it works exclusively via ``cache`` behind the scenes.

``cache`` (and ``get_latest``) enables saving time on instrument communcation, when appropriate:

- in the context of a measurement, the ``get``/``__call__`` interface is the same as of the ``Parameter``)
- also when obtaining parameter's snapshot (summary of it's state)

``_Cache`` provides:

- ``get``/``__call__`` interface, similar to parameter
- ``set`` method, it does the same transofrmations as the parameter's ``set`` method BUT it doesn't perform ``set``'s custom behavior, i.e. communication to the instrument, it just updates the value in the memory in the correct way
- ``timestamp`` of the moment the value has been last updated

Parameter cache suggests a concept of valid/invalid cache (not 100% implemented yet):

- if the value in the cache is *valid*, then it's correct to return it on ``cache.get()`` call
- if the value in the cache is *NOT valid*, then *perhaps* it's more correct to call ``parameter.get()`` and return the "new" actual value

For now in QCoDeS, there are two reasons why/how the value in the parameter's cache may be invalid:

- the parameter has never been set/gotten
  - the implementation checks for ``timestamp`` of the ``cache`` being ``None``
  - cache's ``get`` method has ``get_if_invalid=True`` flag that controls this behavior
    - cache's ``__call__`` method always calls ``get`` with ``get_if_invalid=True``
- ``max_val_age`` amount of time has passed since the value in the cache has been updated
  - if ``max_val_age`` has been set at parameter's ``__init__`` to a not-``None`` value
  - ``max_val_age`` is a property on ``_Cache``

```python
parameter = Parameter(...)

parameter.get()  # the value WILL ALWAYS BE obtained

parameter.cache.get()  # this may NOT underneath call ``parameter.get()``
```

Implicitly (not explicitly), ``cache`` is only used for the snapshot feature.

#### Value and raw value

Parameter distinguishes between ``value`` and ``raw_value``:

- ``value`` is what ``set``/``get`` methods operate with
  - this is the **user-facing** value
- ``raw_value`` is what ``set_raw``/``get_raw`` (and ``set_cmd``/``get_cmd``) operate with
  - frequent example - ``raw_value`` is **instrument-facing** value
  - ``raw_value`` is equal to ``value`` if builtin conversions are not used (offset/scale/val_mapping/get_parser/set_parser)

Validators - ``qcodes.utils.validators``, ``.validate(value)``

```python
p = Parameter(...
   vals=...,
   set_parser=...,
   get_parser=...,
   step=...,
   offset=...,
   scale=...,
   val_mapping={...}  # use exclusively with ``vals``
   )
```

Below is the summary of ``set`` method's behavior, with distinction between ``value`` and ``raw_value``:

```python
def set_wrapper(value: ParamDataType, **kwargs: Any) -> None:
    self.validate(value)

    for step_index, val_step in enumerate(steps):
        self.validate(val_step)

        raw_val_step = self._from_value_to_raw_value(val_step)

        # <.. Check if delay between set operations is required ..>

        set_function(raw_val_step, **kwargs)

        # <.. Check if any delay after setting is required ..>

        self.cache._update_with(value=val_step,
                                raw_value=raw_val_step)
```

Below is the summary of the ``_from_value_to_raw_value`` method that implements parameter's built-in conversions:

```python
def _from_value_to_raw_value(
        self, value: ParamDataType
    ) -> ParamRawDataType:

    # <.. Convert value using val_mapping dictionary ..>
    
    # <.. Apply scale ..>

    # <.. Apply offset ..>

    # <.. Apply set_parser ..>

    return raw_value  # type: ParamRawDataType
```

The summary implementation for ``get`` method is exactly symetrical, but without accounting for ``step``, ``post_delay`` and ``inter_delay``.

#### Snapshot

Parameter is inhereting from ``Metadatable`` class, hence implements ``snapshot`` method.

Parameter's snapshot includes all information about, including it's value:

In [54]:
from qcodes.instrument.parameter import Parameter
from qcodes.utils.validators import Numbers

p = Parameter('p',
              set_cmd=None,
              label='Periodicity', unit='a.u.',
              vals=Numbers(min_value=0, max_value=42),
              scale=2, offset=3,
              inter_delay=0.1, post_delay=0.4
             )

In [55]:
pprint(p.snapshot())

{'__class__': 'qcodes.instrument.parameter.Parameter',
 'full_name': 'p',
 'inter_delay': 0.1,
 'label': 'Periodicity',
 'name': 'p',
 'offset': 3,
 'post_delay': 0.4,
 'raw_value': None,
 'scale': 2,
 'ts': None,
 'unit': 'a.u.',
 'vals': '<Numbers 0<=v<=42>',
 'value': None}


In [56]:
p.set(7)
# Now ``raw_/value`` and timestamp (``ts``) will be not-``None``
pprint(p.snapshot())

{'__class__': 'qcodes.instrument.parameter.Parameter',
 'full_name': 'p',
 'inter_delay': 0.1,
 'label': 'Periodicity',
 'name': 'p',
 'offset': 3,
 'post_delay': 0.4,
 'raw_value': 17,
 'scale': 2,
 'ts': '2020-04-23 17:37:14',
 'unit': 'a.u.',
 'vals': '<Numbers 0<=v<=42>',
 'value': 7}


Parameter implements a few flags to adjust the content and behavior of the snapshot:

- ``snapshot_get=True``
  - ``False`` prevents any update to the parameter (``.get()`` call) during a ``.snapshot()``
  - Useful if the update is time consuming
- ``snapshot_value=True``
  - ``False`` prevents parameter value to be included in the snapshot
  - Useful if the value is large, or not relevant
- ``snapshot_exclude=False``
  - ``True`` allows parent object to prevents parameter from being included in the instrument's snapshot
  - Useful if there are many of the same parameters which are clogging up the snapshot

Why adjusting snapshot's contents/behavior? - saving up on acquiring value and snapshot size.

#### More sophistication

Parameters are not always of "scalar numerical simple-set-get-command" kinds:

- QCoDeS has first-class support for complex-valued parameters
- Non-numerical types are also valid - e.g. strings, predefined constants, ON/OFF, True/False
- Sometimes mixed types - e.g. ``None``+numbers, numbers+strings
- Multiple parameters that are set via single string command at once 
  - ``qcodes.instrument.group_parameter.GroupParameter``
- Non-scalars:
  - tuples of values - e.g. X/Y from a lock-in measurement
  - arrays - e.g. measurement traces from instruments
    - frequently together (at least in relation to) setpoint parameters
- And more, for example:
  - parameters of different meaning that an instrument returns from a single acquisition - e.g. vector network analysers, digitizer cards, e.g. ``MultiParameter``.

So far in many cases, it was possible to reuse the parameter abstraction. However, sometimes it is tricky.

#### ``DelegateParameter``

``DelegateParameter`` (from ``qcodes.instrument.parameter``) is intended for proxy-ing other parameters:

- It **forwards** its ``get``/``set`` to the underlying source parameter
- It **"mirrors"** ``cache`` of the underlying source parameter, delegate does NOT maintain it's own cache
- It allows to specify **different** label/unit/etc

This is great for use cases where the parameter that you'd like to use does not have name/label/unit (also scale/offset) that fit well with the measurement context.

In [41]:
from qcodes.instrument.parameter import Parameter, DelegateParameter

source = Parameter('voltage', 'Voltage', 'V', set_cmd=None)
# say, from some generic instrument

# measurement-context-aware parameter
delegate = DelegateParameter('bias', source, label='Bias voltage', unit='mV', scale=1e3, offset=3.5)

In [43]:
source.set(5)
print(delegate.get())

0.0015


It is possible to extend a ``DelegateParameter`` by overwriting it's ``set_raw``/``get_raw``, but due to cache mirroring that will only work if the conversion between values of the source and delegate parameters is a bijection - that is, the conversion is reversible.

#### Array valued parameters

``ParameterWithSetpoints`` is intended for array-values parameters - say, a call to the instrument returns an array of values. Moreover, some typical measurements that return an array of values, e.g. time trace, can be wrapped into ``ParameterWithSetpoints``.

More here [Parameters/Simple-Example-of-ParameterWithSetpoints](Parameters/Simple-Example-of-ParameterWithSetpoints.ipynb).

Note that ``ArrayParameter`` is legacy, and ``ParameterWithSetpoints`` replaces it.

### Snapshot

Snapshot is a "photograph" of a state of a measurement setup or its piece, be it ``Station``, ``Instrument``, ``Parameter``, or other components.

Snapshot is **not** maintained in memory. Instead, each snapshot-able component can always return/produce its snapshot.

#### Subclass ``Metadatable``

Snapshot-able objects should inherit from ``qcodes.utils.metadata.Metadatable`` and should override ``snapshot_base`` method to return a dictionary representing the state of the object. Then, calling ``snapshot()`` method will return the snapshot of the object.

In [21]:
from qcodes.utils.metadata import Metadatable


class TeamsStatus(Metadatable):
    def __init__(self, name: str, status: str, metadata=None):
        super().__init__(metadata=metadata)
        self.name = name
        self.status = status

    def snapshot_base(
            self, update: bool = False,
            params_to_skip_update: Optional[Sequence[str]] = None) -> Dict:
        return {'status': self.status}


teams_status = TeamsStatus('Fridge25 status', 'busy')

pprint(teams_status.snapshot())

{'status': 'busy'}


####  ``metadata`` argument

Note that ``Metadatable`` also implements ``metadata`` keyword argument - ``metadata`` is expected to be a dictionary, and it will be included in the snapshot automatically (if it is not ``{}``).

In [22]:
favorites = Metadatable(metadata={'number': 6 + 9j})
favorites.load_metadata({'fruit': 'manadrine'})  # ``dict.update`` behavior

In [23]:
pprint(favorites.snapshot())

{'metadata': {'fruit': 'manadrine', 'number': (6+9j)}}


#### Add  ``Metadatable`` to station

``Metadatable`` objects can be added to the ``Station``, other objects - should not (otherwise ``snapshot()`` will raise).

In [24]:
station = Station(teams_status)
station.add_component(favorites)

'component1'

In [25]:
pprint(station.snapshot())

{'components': {'Fridge25 status': {'status': 'busy'},
                'component1': {'metadata': {'fruit': 'manadrine',
                                            'number': (6+9j)}}},
 'config': None,
 'default_measurement': [],
 'instruments': {},
 'parameters': {}}


#### Create ``JSON`` snapshot

Although snapshot is a python dictionary, it is always meant to be converted to JSON and stored next to the measured data. QCoDeS maintains it's own JSON converter class, ``qcodes.utils.helpers.NumpyJSONEncoder``, to support conversion of objects that are frequently met in measurement world, such as ``numpy`` arrays, complex numbers, pickle-ables.

In [26]:
from qcodes.utils.helpers import NumpyJSONEncoder
import json

json_snapshot = json.dumps(
    station.snapshot(), cls=NumpyJSONEncoder,
    indent=2, sort_keys=True  # for pretty-printing
)

In [27]:
print(json_snapshot)

{
  "components": {
    "Fridge25 status": {
      "status": "busy"
    },
    "component1": {
      "metadata": {
        "fruit": "manadrine",
        "number": {
          "__dtype__": "complex",
          "im": 9.0,
          "re": 6.0
        }
      }
    }
  },
  "config": null,
  "default_measurement": [],
  "instruments": {},
  "parameters": {}
}


#### ``update`` arguments

```
snapshot(update=True/False)
```

Note that ``snapshot`` has an ``update`` kwarg, and ``snapshot_base`` has ``update`` and ``params_to_skip_update`` kwargs - those allow to optimize querying the state of a ``Metadatable`` object.

Passing ``update=True`` indicates a forceful refresh of the object's knowledge of it's state. Parameter is a great example:

- with ``update=False`` - the value from the cache will be used instead of ``get``-ting it, i.e. requesting from an instrument
- with ``update=True`` - parameter's value will be gotten for sure, thus updating the cached value, and resulting in a more up-to-date snapshot

``update`` **default** value for ``snapshot`` is ``False``, but for ``snapshot_base``:

- ``False`` for ``Metadatable``
- **``True``** for ``Station`` 
- **``True``** for ``VisaInstrument``
- ``False`` for ``InstrumentBase``
- **``True``** for ``_BaseParameter``

Note that ``_BaseParameter`` implements flags that can overrule ``update`` argument behavior.

```python
p -> snapshot_get=False

p.snapshot(update=True)
```

``snapshot_base`` has another argument - ``params_to_skip_update``. This argument so far implemented only for instruments - it is useful to pass a list of parameter names which are slow to update but can be updated in a different way (example, QDAC).

### Measurement code

Measurement code is code that works with instruments, parameters, and other methods/functions to run a measurement which includes controlling hardware, acquiring data, and storing data.

Historically data aquisition frameworks have been struggling to provide balance between flexibility and convenience.

- Flexibility of the framework should allow experimentalists to not limit their progress when they have a measurement in mind.
- Convenience of the framework should allow measurement ideas to become measurement code in a short time.

Both these requirements are extremely hard to fulfill. QCoDeS has a legacy ``Loop`` feature that was a great convenience and was flexible enough for quite many straightforward measurements, but would becomes difficult and hacky for other measurements.

At the moment, QCoDeS is priorotizing "flexibility" by taking the following approach - **allow freeform python** as measurement code but providing clear API for data storage.

For **very frequently** performed measurements, QCoDeS is willing to implement and support reasonably flexible "conveniences", for example, ``do1d``/``do2d`` from ``qcodes.utils.dataset.doNd``.

It is known that some enthusiasts do put their effort into solving the nontrivial software engineering problem of coming up with more flexible AND convenient measurement code frameworks, for example, ``github.com/damazter/pysweep``.

## I wanna STORE

How and where do I store what I measured? And access back?

Measurements, experiments, datasets, parameters? A DATABASE?

### Storing what is measured

Instruments and parameters provide great abstractions to perform measurements: ``set`` this, ``get`` that - great. The question is now how to store the measured data.

What?

- data, the values themselves
- metadata about the values - label, unit, etc
  - structure of the measured data, if possible
- station snapshot
- mesaurement meta information

How?

- persiting data ASAP
- flexible, allowing basically any structure of measured data
- data and metadata conveniently linked
- efficiently, compared to characteristic measurement times
- not allow overwriting to prevent mistakes

This is what ``Measurement``, ``DataSaver`` and ``DataSet`` are intending to implement.

### API overview

In short, this is how the API looks (skipping details):

```python
measurement = Measurement(...)
...
with measurement.run() as datasaver:
    ...
    datasaver.add_result(...)
    ...
...
dataset = datasaver.dataset
...
```

``Measurement`` object from ``qcodes.dataset.measurements`` is the front interface for running measurements with QCoDeS. It's primary purpose is to capture metadata about the measurement, and provide a context manager that explicitly scopes the measurement routine.

``run()``-ning a ``Measurement`` creates a ``Runner`` context manager. Upon entering the context, ``Runner`` will initiate a ``DataSaver`` object and thus the underlying storage mechanism, and at exiting it will do necessary operations expected at completion of the measurement routine. It is at entering the context, when a new ``DataSet`` object is being created with the necessary metadata about the measurement such as parameters and snapshot.

``DataSaver`` object provides interface for passing in the acquired data. It validates the data and forwards it downstream to the storage mechanism. ``DataSaver`` object also holds an instance of the created ``DataSet`` object (from ``qcodes.dataset.data_set``) which can be used to accessed the stored measurement data and its metadata.

### ``Measurement``

Let's get into details of the ``Measurement`` object.

#### Experiment, name, station

``Measurement`` takes the following arguments for its initialization:

- ``Experiment`` - an object that repesents ``exp_name`` and ``sample_name`` (from ``qcodes.dataset.experiment_container``)
  - if not passed, the "latest" experiment is taken
- ``name`` - name for the measurement, and hence the resulting ``DataSet``
- ``Station`` - the station object, in order to take the snapshot of the state of the station, and store it next to the measured data
  - if not passed, ``Station.default`` is used conveniently

#### Registering parameters

``Measurement`` provides an interface to declare the intent of a measurement, in particular, it's structure.

This information provides means to reason about the stored data - how the values relate to each other. This information comes in very handy when:

- storing data, especially, ``DataSaver.add_result``
- accessing data (aka loading), for example, ``DataSet.get_parameter_data``
- processing data, for example, for visualization.

That interface has two parts - one that is not aware of QCoDeS parameters, and the other one that is. The former part makes the ``Measurement`` flexible and independent, and the latter makes it convenient to use. It is reasonable to say that the ``Measurement`` object bridges the QCoDeS parameters world with the QCoDeS ``DataSet`` world, thus making the two worlds independent.

##### Parameter relationships

Measurement structure is defined via its parameters and relationships between them. Parameters can be:

- independent, aka setpoints
- dependent, aka measured
- standalone

This information allows to reason about data as a collection of **parameter trees** - roots of a tree are independent parameters that the main stem, the dependent parameter, depends on.

For example, in the image below:

- A depends on x and y
- B depends on x and y - notice that it's a separate tree
- C depends only on x
- t is a standalone

![param-trees](param_trees.svg)

**NOTE**: there is another parameter relationship in QCoDeS that exists, but is not used for any functionality - inference. It is possible to declare that "parameter A is inferred from parameter B", this information will be recorded, but will not be used anywhere in QCoDeS. This feature will either be removed or improved if deemed needed.

##### ``register_parameter``

``Measurement`` exposes a ``register_parameter`` method. It is intended to be called with QCoDeS parameters - ``_BaseParameter``, ``Parameter``, their subclasses, including some of the more interesting ones.

``register_parameter`` does the following:

- extracts name/label/unit from the given parameter
- notes what parameters the given one depends on, aka "setpoints"
- notes ``paramtype`` which will be helpful for data storage later

For some of the ``_BaseParameter`` subclasses, ``register_parameter`` digs deeper into the given parameter to extract additional info on its dependencies. The supported subclasses are ``ParameterWithSetpoints``, ``ArrayParameter``, ``MultiParameter`` - all these represent at least a branch of a parameter tree (unlike ``_BaseParameter`` itself representing only a leaf).

For example, if a ``ParameterWithSetpoints`` is registered, then its ``.setpoints`` parameters are registered automatically and correctly, as setpoints of the parameter itself.

A companion ``register_custom_parameter`` method takes parameter name/label/unit/paramtype/setpoints explicitly, in other words, without the need for a QCoDeS parameter.

##### ``paramtype``

QCoDeS differentiates between the following data types: ``numeric``, ``array``, ``text``, ``complex``.

These allow the data storage to implement optimizations based on this information. For example, with QCoDeS default (and only) data storage implementation (based on ``sqlite``), saving large amounts of numerical data as ``array`` paramtype can be **much** faster than as ``numeric``. More on this later.

##### Behind the scenes

Behind the scenes ``Measurement`` object accumulates the information about the parameters and their relationships in order to later provide it to the ``DataSet`` that is to be created when the measurement will be run.

``Measurement`` and ``DataSet`` share the way the parameter trees are represented:

- ``ParamSpecBase`` object represents one parameter
  - it only has the 4 fields - name/label/unit/paramtype
- ``InterDependencies_`` object represents one or more parameter trees
  - in other words, represents dependency relationships between ``ParamSpecBase`` objects
  - can only be **valid**, in other words, "validation" is built-in
  - convenient interface for interactive with parameter trees

Below is a small example for the two objects:

In [28]:
from qcodes.dataset.descriptions.param_spec import ParamSpecBase
from qcodes.dataset.descriptions.dependencies import InterDependencies_

x = ParamSpecBase('x', 'numeric', 'Distance', 'm')
t = ParamSpecBase('t', 'numeric', 'Time', 's')

deps = InterDependencies_(dependencies={x: (t,)})

print(deps)

InterDependencies_(dependencies={ParamSpecBase('x', 'numeric', 'Distance', 'm'): (ParamSpecBase('t', 'numeric', 'Time', 's'),)}, inferences={}, standalones=frozenset())


Note that ``InterDependencies_`` object becomes part of a bigger container, the ``RunDescription`` object, which gets serialized upon storage.

``RunDescription``, ``InterDependencies_`` and ``ParamSpecBase`` live in ``qcodes.dataset.descriptions.*``.

### ``DataSaver``

Now let's look at ``DataSaver`` - it's primary purpose it to provide an interface for passing "new data to store" and implement such an interface by validating the input data and passing that to the storage mechanism, in essense, to ``DataSet``.

#### Relation to dataset

``DataSaver`` holds a reference to the ``DataSet`` that is being populated with data within the measurement context. To get access to that ``DataSet``, just use the ``.dataset`` property of ``DataSaver``.

The API assumes that the users:

- will not interact with the ``DataSet`` within the measurement context
- will not interact with the ``DataSaver`` outside the measurement context

#### ``add_result``

``DataSaver.add_result`` is the interface for "submitting" data for saving/storing.

##### Message-based

The ``DataSaver.add_result`` in principle looks like a message-based interface. Each "result" is a "message" that tells the world that "a new piece of data has been acquired". This approach enables rich flexibility for the structure of measurement code.

The challenge of this approach is to reconstruct a meaningful representation of the data after the acquisition is finished (i.e. "is data on a regular grid?"). This challenge is addresssed from two angles: automated heuristics, and adding more helpful metadata upfront (if possible).

##### Inputs

Users are expected to provide ``(name/parameter, value(s))`` tuples. The first element is either a string representing a name of a registered parameter or a parameter instance. The second argument is values:

- a ``float``/``int``/``string``/``complex`` value
- a ``list``/``tuple``/``numpy``-array of those for convenience
- an array - a ``numpy``-array or ``list``/``tuple`` of values

##### Validation

This flexibility of the interface together with the dynamic nature of python calls for strict and helpful validation - this is the main part of what ``add_result`` does. And it does this based on the information about parameter trees that hass been given to ``Measurement``.

The following are the rules which ``add_result`` validates against:

- All provided **parameters MUST be registered** via ``Measurement``
- For each dependent parameter values, the values of its **setpoints MUST be provided**
  - It is possible to provide parameters **in any order**, e.g. dependent parameter before its setpoints
- **Data type** of the provided values **MUST match** the types supported by the ``paramtype`` of the parameter
  - integer/float for "numeric", complex for "complex", str for "text", integer/float/complex for "array"

- Provided lists/tuples/numpy-arrays of **values MUST have consistent shapes**
  - e.g. for a dependent parameter and its setpoints, number of provided values in the lists should be the same
    - Passing **scalar value for a setpoint** parameter(s) when a list/tuple/numpy-array was passed for a dependent parameter **is possible** - the value of that setpoint parameter is assumed to be the same and equal to that scalar values for all the values in the list for the dependent parameter.

Behind the scenes this validation is implemented largely based on:

- interaction with ``Interdependencies_`` object of the created ``DataSet`` that ``DataSaver`` has access to
- ``numpy`` interfaces for types and shapes - all inputs of ``add_result`` internally get converted to ``numpy`` objects for the convenience of validation

##### And then?

After validation is done, ``DataSaver.add_result`` prepares the values to be passed to ``DataSet.add_results`` (yes, confusing names). Although, ``DataSet.add_results`` is not a private method, it is NOT supposed to be used by users. The ``DataSet`` then takes care of actually storing the data (aka "results").

### ``DataSet``

``DataSet`` (from ``qcodes.dataset.data_set``) is final destination of the acquired data and meta information around it. It is also the primary interface for the user for accessing the data.

#### User-facing API

``DataSet`` exposed the following user-facing APIs:

- *Identity*:
  - ``guid``
  - ``captured_run_id``, etc

- *Meta information*:
  - ``name``, ``sample_name``, ``exp_name``
  - ``path_to_db`` (also ``conn``ection to DB)
  - ``description`` (``RunDescriber``)
  - *Parameters*:
    - ``interdeps``
    - ``dependent_parameters``
  - ``snapshot``
  - *Timestamps*
  - *Metadata (key-value pairs)*:
    - ``metadata``, ``get_metadata``, ``add_metadata``

- *Data*:
  - ``get_parameter_data``
  - ``number_of_results`` / ``__len__``
  - *Convenient*:
    - ``get_data_as_pandas_dataframe``
    - ``write_data_to_text_file``

Some API have also ``*_raw`` versions, for example, ``snapshot`` and ``snapshot_raw`` where "raw" is how the piece of information is represented in the underlying storage.

``DataSet`` also has deprecated/not-recommended API:

- for data access - ``get_data``, ``get_setpoints``, ``get_values``
- for parameters - ``parameters``, ``paramspecs``

#### State

``DataSet`` object can be in three states:

- **pristine**
  - when dataset is just created
  - that's the moment to add information about parameters, etc.
  - it is not possible yet to add data to a dataset
  - ``Runner`` creates the dataset in this state, fills it with information, and marks it for the next state
  - ``mark_started`` moves the dataset to the next state

- **running**
  - when dataset is ready for adding data to it (aka "results")
  - it's not possible to change meta information anymore
  - ``DataSaver`` adds data to a dataset in this state
  - ``mark_completed`` moves the dataset to the next state

- **completed**
  - when adding data to dataset is done/finished
  - it's not possible add data anymore
  - this is the final state of the dataset
  - users are expected to work with a dataset in this state

These are implemented on top of two flags, ``started`` and ``completed``. ``pristine`` and ``running`` are convenience/clarity flags.

#### Metadata feature

``DataSet`` exposes a metadata feature for attaching key-value pairs of information to it. This feature has a flaky implementation from the storage side, and the interface also calls for clarification. Due to these challanges this feature is not advertised at the moment, which should not deminish it's potential value.

At the moment it's interface is to be used only like this:

* ``add_metadata("key", "value")`` method
  - actively saves metadata to persistent storage
  - "key" should be a string, "value" should be a simple data type like numeric/text
  - calling more than once **updates** the value of the key
* ``get_metadata("key")`` method
  - actively loads from persistent storage
* ``metadata["key"]`` property
  - works "as expected" ONLY on getting keys
  - setting keys does NOT persist them, use ``add_metadata`` instead

#### Plotting

Plotting of a ``DataSet`` is implemented by ``plot_dataset`` function from ``qcodes.dataset.plotting`` module. It's purpose is to provide decent default plotting which is good enough for straightforward 1D/2D data.

It is important to note what are ``plot_dataset`` features as they are frequently expected by experimentalists:

- uses ``matplotlib``
- uses labels/units for axes
- only 1D and 2D plots (gridded and scatter) supported
- rescaling axes to engineering prefixes, e.g. ``0.00000005 -> 50 n``
- default rasterizing for 2D plots
- adjust colorbar scale to disregard outliers
- clipping plotted data via cutoff percentiles

### SQLite database

QCoDeS uses an ``sqlite`` database as it's storage backend for storing acquired data. Let's get into why and how. 

#### Local measurement log book

The idea of a database comes from the intent of bringing order for experimentalists:

- store measurement data in one place
- store the data together with relevant metadata
- help users with bookkeeping
- browsing through performed measurements
- etc.

All these can be summarized into as a "measurement log book".

This lead to a single DB approach. QCoDeS API is following it. Reason - for the user to have "one" place where all the experimentals data is stored (and can't be lost ;) ).

The path to the default database is stored in QCoDeS config, many functions/objects use this as a fallback when no explicit database path is provided (for example, ``load_by_*`` functions).

#### DB structure

The database stores information about datasets, aka measurement "runs". In the current design, the database also stores the measured data (not only metadata).

The QCoDeS SQLite database consists of the following tables:

- **``runs``** - stores most of meta information about datasets
  - name, timestamps, parameters, snapshot, etc
  - includes reference to an ``Experiment`` via ``exp_id``
  - includes reference to a "result" table with raw data
- **``experiments``** - stores ``exp_name``/``sample_name`` (+extras)
- **"results" tables** - that's where the raw measurement data is stored, 1 table per 1 dataset
- ``layouts``/``dependencies`` tables - deprecated/unused - used in the past to store parameters and their relationships

##### SQLite choice

``SQLite`` was chosen largely because:

- it's lightweight, fast, and simple, yet powerful
- great choice for an in-application database
- built-in support in python via ``sqlite`` module

##### Runs table

``runs`` table is the main table of QCoDeS database. It lists all the runs (aka datasets) with their meta information.

``run_id`` is this table's primary key. Because of it's simple-to-use, users started using it to refer to runs, but it is not robust against moving runs between different database files, hence ``captured_run_id`` was introduced.

Datasets ``guid`` is just a column. The fact that same database file cannot have more than one dataset with the same GUID is taken care of by QCoDeS, and NOT the database schema.

Linkage to an experiment, and hence ``exp_name``/``sample_name``, is established via ``exp_id``. ``exp_id`` is the primary key of the ``experiments`` table.

Information about parameters is stored in ``run_decription`` as a JSON representation of ``RunDescriber`` object. ``parmaeters`` column and ``layouts`` and ``dependencies`` tables are unused/deprecated. Note that ``RunDescriber`` hsa versions - at the moment, version 0 is stored in the database, and version 1 is used on QCoDeS API level.

Snapshot is stored as JSON in ``snapshot`` column.

``metadata`` feature of ``DataSet`` object is implemented via creating new columns in the ``runs`` table for each new "key" and storing the "values" in that column. This makes for a "dynamic schema" of the database. Depending on the future of the  ``DataSet.metadata`` this may change. Note that due to historical reasons, ``snapshot`` column of the table is accessed via metadata-related API in ``qcodes.dataset.sqlite.queries``.

``results_table_name`` table contains the name of the table within the same database file where the raw data for this run/dataset is stored. The name of this table is composed from the ``name``, ``exp_id`` and ``result_counter`` values and a format string defined in ``experiments`` table for the experiment with given ``exp_id`` (default is ``"{name}-{exp_id}-{result_counter}"``).

##### Experiments idea

Experiment was meant as a bucket for datasets/runs for better organization. It hasn't been really used as such, instead ``sample_name`` and ``exp_name`` became important identification attributes for datasets.

From QCoDeS API apoint of view - a database HAS to contain an experiment before a dataset can be even created, and ``Measurement`` expects an instance of ``Experiment`` in order to be able to attach new ``DataSet``s to correct ``sample_name``/``exp_name``. Conveniences such as ``load_or_create_experiment`` and ``get_last_experiment`` had to be created to help users.

There are ideas to deprecate this API or improve it by providing clear "active experiment".

From the database structure point of view, it's unclear whether having a separate experiments table is better than having ``exp_name``/``sample_name`` as explicit columns in the ``runs`` table.

##### Results tables

Each dataset/run has it's own "results" table.

**The table has an "id" column**, and a column for each parameter of the dataset. "id" column stores the order of "results" coming in from ``DataSaver.add_result``.

**Numeric and text data types** (``"numeric"`` and ``"text""`` ``paramtypes``) are stored "as-is", which means that SQLite engine can interact with those values, for example, potentially allowing queries with ``WHERE`` clauses.

**Complex numbers** are stored as binary blobs, even for scalar values. The ``numpy`` array binary representation is used for that.

**Arrays are stored as binary blobs** (for ``"array"`` ``paramtype`` parameters), those blobs are stored in the cells of the "results" tables. The ``numpy`` array binary representation is used for that.

| id | x | A    |
|----|---|------|
| 1  | 2 |*BLOB*|

**Data is stored with parameter trees in mind**. That means that if two dependent parameters are dependent on the same independent parameter, then the values of the dependent parameters will be duplicated and SQLite's ``NULL``s will be used to fill the empty gaps.

In the example below, parameters A and B both depend on parameters x and y:

| id | x | y | A    | B    |
|----|---|---|------|------|
| 1  | 4 | 5 | 9    |*NULL*|
| 2  | 4 | 5 |*NULL*| 8    |

In earlier QCoDeS releases (~0.2.0), the data from the example was stored as presented below:

| id  | x   | y   | A   | B   |
|-----|-----|-----|-----|-----|
| 1   | 4   | 5   | 9   | 8   |

From those times, QCoDeS ``DataSet`` still has unrecommended ``get_data``/``get_values``/``get_setpoints`` methods. The problem with those is that they return raw data in the same was as it is presented in the "results" table, thus NOT accounting for parameter trees.

``get_parameter_data`` method in turn DOES take into account the parameter trees and ALWAYS returns correctly shaped raw data. ``get_parameter_data`` is aware of the array blobs and performs correct unraveling when needed based on the information about parameter trees.

Unrecommended API is being deprecated in QCoDeS 0.14.0 (unreleased yet).

#### Data saving optimizations

Experimental/measurement environment is full of requirements on efficiency. The following features of QCoDeS SQLite database are adressing those.

- **WAL mode** - QCoDeS database uses SQLite's Write-Ahead-Log (WAL) mode.
  - It enables 1 "write" and multiple "reads" simultaneously
  - Slightly improves read/write performance of the database
  - Two more files are present next to ``.db`` file when it's active

- **Storing "array"s as binary blobs** - ``paramtype`` ``"array""`` data is stored as opaque binary blob in table cells
  - SQLite supports storing binary blobs out of the box
  - QCoDeS stores array data as ``numpy`` binary blobs
  - For large data acquisitions this brings great save/loading efficiency as compared to storing individual numbers, and comes close to storage solutions like ``HDF5``
    - However, the efficiency depends on how the dataset is structured

- **Writing data every ``write_period``** - accumulating "results" before "flushing" them to the database
  - enables to balance cumulative data acquisition overheads with data storage overheads
  - ``write_period`` is settable on ``Measurement``
  - ``DataSaver`` is respondible for accumulating "results" and calling ``DataSet.add_results`` every ``write_period`` seconds

- **Write data in background thread** - ``write_in_background`` flag enables writer thread for given measurement
  - The I/O-bound SQLite interaction can happen in parallel with data acquisition
  - Potentially saving up to 50% of the measurement time as compared to acquiring and storing sequentially
  - at the moment, support is only within a given ``Measurement`` context

#### Design challenges

Due to historical reasons, it has been challenging to develop the "store" part of QCoDeS.

* QCoDeS API and the QCoDeS' SQLite "storage backend" are coupled quite closely, ``DataSet`` and other objects are leaky abstractions.

* Database schema and database-related part of QCoDeS codebase has abundance of deprecated/unused features and interfaces, it is sometimes unclear what is private and what it public to users.

* QCoDeS database may be combining different approaches to database design, e.g. versioning of run_description separately from versioning of the database schema, e.g. key-value store vs "relational" store

* Support for different storage backends is difficult because QCoDeS does not define an interface that would separate ``DataSet`` API from the storage (however, some prototypes of "data storage interface" have been tried).

## I want MORE

What else does QCoDeS provide out of the box? Quite many things (and many more to come). Below are the topics for self-exploration (they are either clear from the code, or are documented reasonably well):

- *features*
  - Config
  - Plotting support
  - Before/after actions of Measurement
  - DataSet subscribers
  - Linking datasets / parent datasets
  - logging

- *parameters*
  - inter_delay/post_delay
  - Multiparameter
  - Group parameter
- *instruments*
  - Instrument's functions
  - writing drivers
  - contrib-drivers

- *infra*
  - DelegateAttributes
  - Deprecation
  - versioning infra for of RunDescriber
- *database*
  - structure of sqlite module
  - versioning infra for database
  - Extracting runs/datasets from db to another

- *extras features*
  - Monitor
  - Interactive widget
- *Legacy API*
  - ParamSpec
  - Loop-entangled features